In [ ]:
import requests
from urllib.parse import urlencode

In [ ]:
Google_api_key='enter api key '

## Google maps client API

In [ ]:


class GoogleMapsClient(object):
    lat = None
    lng = None
    data_type = "json"
    location_query = None
    api_key = None
    
    def __init__(self, api_key=None, address_or_postal_code=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if api_key == None:
            raise Exception("API key required")
        self.api_key = api_key    
        self.location_query = address_or_postal_code
        if self.location_query != None:
            self.extract_lat_lng() 
    
    def extract_lat_lng(self, location =None):
        loc_query = self.location_query
        if location != None:
            loc_query = location
        endpoint = f"https://maps.googleapis.com/maps/api/geocode/{self.data_type}"
        params ={"address" : loc_query, "key": self.api_key}
        url_params = urlencode(params)
        url = f"{endpoint}?{url_params}"
        r = requests.get(url)
        if r.status_code not in range(200,299):
            return {}
        latlng = {}
        try :
            latlng = r.json()['results'][0]['geometry']['location']
            #return r.json()['results'][0]['geometry']['location']
        except:
            pass#passing coz in somecases successful lookup but wont find data
        lat, lng = latlng.get('lat'), latlng.get('lng')
        self.lat = lat
        self.lng = lng
        return lat, lng


    def nbsearch(self, keyword= "keyword",radius= 5000, location = None):
        lat, lng = self.lat, self.lng
        if location != None:
            lat, lng = self.extract_lat_lng(location)
        endpoint = f"https://maps.googleapis.com/maps/api/place/nearbysearch/{self.data_type}"
        params = {
            "key": self.api_key,
            "location":f"{lat},{lng}",
            "radius": radius,
            "keyword" : keyword

        }
        params_encoded = urlencode(params)
        places_url= f"{endpoint}?{params_encoded}"
        r = requests.get(places_url)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    
    def detail(self, placeid):
        place_id= placeid
        url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name%2Crating%2Cformatted_phone_number&key={self.api_key}"

        payload={}
        headers = {}

        r = requests.request("GET", url, headers=headers, data=payload)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    

In [ ]:
client = GoogleMapsClient(api_key= Google_api_key,  address_or_postal_code= "location")
print(client.lat, client.lng)


##Place API

In [ ]:
client.nbsearch("search feed")

In [ ]:
client.detail("placeid")#place_id